In [1]:
import sys
sys.path.append('/opt/conda/pkgs')
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4262b1bf4bf1ffb403c0eb7a42ad5906_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/4506eccf78279d93d0e8a34c035e91c5_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/6bda807e3967eae797c7b1b9eeaee8db_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c2a47d89d1d34e789fdf782557bb7194_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/c6c5514ada15b890fb27d1e36371554c_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/d964a294c2d0fef56a434c021026281e_/lib/python3.12/site-packages")
sys.path.append("/home/miaocj/docker_dir/kNN-overlap-finder/.snakemake/conda/e1c932db5cd4271709e54d8028824bc9_/lib/python3.12/site-packages")
import gzip, json
from Bio import SeqIO
import scipy.sparse as sp
from collections import Counter
import numpy as np
import pandas as pd
fasta_gz_file = '/home/miaocj/docker_dir/kNN-overlap-finder/data/regional_reads/Ecoli/all/ONT/reads.fasta.gz'
paf_gz_file = '/home/miaocj/docker_dir/kNN-overlap-finder/data/regional_reads/Ecoli/all/ONT/alignment.paf.gz'
with gzip.open(paf_gz_file, "rt") as file:
    max_values = {}  
    for row in file:  
        columns = row.strip().split('\t') 
        query_id = columns[0]  
        match_bases = int(columns[9]) 
        max_values[query_id] = columns 
        if query_id in max_values:  
            if match_bases > int(max_values[query_id][9]):  
                max_values[match_bases] = columns
        else:  
            continue

In [2]:
import sys
import pickle, os, gzip, json, sys, itertools
from pathlib import Path
from importlib import reload
from dataclasses import dataclass, field
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import pandas as pd
import scipy as sp
from typing import Mapping  
import mmh3
from itertools import chain  
import sharedmem

In [3]:
sys.path.append("scripts")
sys.path.append("../../scripts")
from graph import OverlapGraph, GenomicInterval, get_overlap_statistics, remove_false_edges


/home/miaocj/docker_dir/kNN-overlap-finder/scripts/../lib


In [4]:
import numpy as np  
import collections  
from typing import Mapping  
from sklearn.neighbors import NearestNeighbors  
from scipy.spatial.distance import hamming  
import mmh3  
max_n_neighbors = 20
MAX_SAMPLE_SIZE = int(1e9)
k_values = np.arange(2, max_n_neighbors + 1)


In [5]:
npz_path = "/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/human/HLA/ONT_R9/kmer_k16/feature_matrix.npz"
tsv_path = "/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/human/HLA/ONT_R9/kmer_k16/metadata.tsv.gz"
json_path = "/home/miaocj/docker_dir/kNN-overlap-finder/data/feature_matrix/human/HLA/ONT_R9/kmer_k16/read_features.json.gz"

meta_df = pd.read_table(tsv_path).iloc[:MAX_SAMPLE_SIZE, :].reset_index()
read_indices = {read_name: read_id for read_id, read_name in meta_df['read_name'].items()}
feature_matrix = sp.sparse.load_npz(npz_path)[meta_df.index, :]

with gzip.open(json_path, "rt") as f:
    read_features = json.load(f)
    read_features = {i: read_features[i] for i in meta_df.index}

read_ids = np.array(list(read_features))

In [11]:
def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)
    binary_string = "{0:032b}".format(hash_value & 0xFFFFFFFF)  
    hash_array = np.array([int(x) for x in binary_string], dtype=np.int32) 
    hash_array = np.where(hash_array == 0, -1, 1)  
    return hash_array  

def _get_table(
    read_features: list,  
    feature_matrix: list,
    *,
    seed: int,
    tf:bool,
    idf:bool,
    repeat=100) -> Mapping[int,list]:  
    
    rng = np.random.default_rng(seed)  
    hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeat, dtype=np.uint64)  

    kmer_num = feature_matrix.shape[1]
    hash_table = np.empty((kmer_num,repeat),dtype=object)  
    for flag,seed in enumerate(hash_seeds):
        for kmer_index in range(kmer_num):
            hash_table[kmer_index,flag]=_hash(kmer_index, seed=seed)
    return hash_table
def _get_simhash(
    read_features: list,  
    feature_matrix: list,
    hash_table,
    *,
    seed: int,
    tf:bool,
    idf:bool,
    repeat=100) -> Mapping[int,list]:  
    all_read_simhash = []
    if idf == True:
        nested_list = list(read_features.values())
        unrongh_nest = [list(set(sublist)) for sublist in nested_list]  
        merged_list = list(chain.from_iterable(unrongh_nest)) 
        count = Counter(merged_list)
        sorted_counts = dict(sorted(count.items(), key=lambda x: x[0]))  
        times = np.array(list(sorted_counts.values()))
        x = len(read_features)
        arr = np.full(feature_matrix.shape[1],x)  
        idf = np.log(arr/times)
        hash_table = hash_table*idf[:, np.newaxis]
    if tf == False:
        read_features = {k:list(set(v)) for k,v in read_features.items()}

    for k,read_kmer in read_features.items():
        one_read_hash = np.sum(hash_table[read_kmer,:],axis=0)
        conc_hash = np.concatenate(one_read_hash)
        simhash = np.where(conc_hash > 0, 1, 0)
        all_read_simhash.append(simhash)
    reads_simhash_array = np.array(all_read_simhash)

    return reads_simhash_array 

hash_table = _get_table(read_features,feature_matrix,seed = 15232,tf = False,idf=False,repeat=100)
print("1")
reads_simhash_array = _get_simhash(read_features,feature_matrix,hash_table,seed = 15232,tf = True,idf=False,repeat=100)


1


In [35]:
def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)
    binary_string = "{0:032b}".format(hash_value & 0xFFFFFFFF)  
    hash_array = np.array([int(x) for x in binary_string], dtype=np.int8) 
    hash_array = np.where(hash_array == 0, -1, 1)  
    return hash_array  

def _get_table(
    read_features: list,  
    feature_matrix: list,
    *,
    seed: int,
    tf:bool,
    idf:bool,
    repeat=100) -> Mapping[int,list]:  
    
    rng = np.random.default_rng(seed)  
    hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeat, dtype=np.uint64)  

    kmer_num = feature_matrix.shape[1]
    hash_table = np.empty((kmer_num,repeat,32),dtype=np.int8)  
    for flag,seed in enumerate(hash_seeds):
        for kmer_index in range(kmer_num):
            hash_table[kmer_index,flag,:]=_hash(kmer_index, seed=seed)
            new_hash_table=np.reshape(hash_table,(kmer_num,3200))
    return new_hash_table

def _get_simhash(
    read_features: list,  
    feature_matrix: list,
    hash_table,
    *,
    seed: int,
    tf:bool,
    idf:bool,
    repeat=100) -> Mapping[int,list]:  
    all_read_simhash = []
    if idf == True:
        nested_list = list(read_features.values())
        unrongh_nest = [list(set(sublist)) for sublist in nested_list]  
        merged_list = list(chain.from_iterable(unrongh_nest)) 
        count = Counter(merged_list)
        sorted_counts = dict(sorted(count.items(), key=lambda x: x[0]))  
        times = np.array(list(sorted_counts.values()))
        x = len(read_features)
        arr = np.full(feature_matrix.shape[1],x)  
        idf = np.log(arr/times)
        weighted_hash_table = hash_table*idf[:, np.newaxis]
    else:
        weighted_hash_table = hash_table
    if tf == False:
        read_features = {k:list(set(v)) for k,v in read_features.items()}

    for read_kmer in read_features.values():
        one_read_hash = np.sum(weighted_hash_table[read_kmer,:],axis=0)
        simhash = np.where(one_read_hash > 0, 1, 0)
        all_read_simhash.append(simhash)
    reads_simhash_array = np.array(all_read_simhash)

    return reads_simhash_array 

#hash_table = _get_table(read_features,feature_matrix,seed = 15232,tf = False,idf=False,repeat=100)
print("1")
reads_simhash_array = _get_simhash(read_features,feature_matrix,hash_table,seed = 15232,tf = True,idf=False,repeat=100)

1


In [6]:
def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)
    binary_string = "{0:032b}".format(hash_value & 0xFFFFFFFF)  
    hash_array = np.array([int(x) for x in binary_string], dtype=np.int8) 
    hash_array = np.where(hash_array == 0, -1, 1)  
    return hash_array  
def mp_get_hashtable(  
    feature_matrix: list,   
    repeat: int,   
    seed: int,
    processes:int,) -> Mapping[int,list]:  
      
    rng = np.random.default_rng(seed)  
    hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeat, dtype=np.uint64)  
    kmer_num = feature_matrix.shape[1]
    hash_table = np.empty((kmer_num,repeat,32),dtype=np.int8) 

    with sharedmem.MapReduce(np=processes) as pool:

        def work(i):
            seed = hash_seeds[i]
            result = np.empty((kmer_num,32), dtype=np.int8) 
            for kmer_index in range(kmer_num):
                result[kmer_index,:]=_hash(kmer_index, seed=seed)
            return i,result

        def reduce(i, result):
            hash_table[:,i,:] = result            

        pool.map(work, range(repeat), reduce=reduce)
    return hash_table
hash_table = mp_get_hashtable(read_features,feature_matrix,15232,10)

/tmp/ipykernel_2861509/2481070871.py:32: SparseEfficiencyWarning: Comparing a sparse matrix with 0 using == is inefficient, try using != instead.
  hash_table = mp_get_hashtable(read_features,feature_matrix,15232,10)


In [19]:
one_read_hash = np.sum(hash_table[list(read_features.values())[0],:,:],axis=0)
hash_table

array([[20, 10, 12, ..., 36, 8, 4],
       [4, 0, -4, ..., 6, 0, -40],
       [-8, -4, 20, ..., 0, 12, 2],
       ...,
       [-26, -10, 30, ..., 0, 4, -4],
       [-24, 38, 4, ..., -6, 22, -22],
       [2, 14, 28, ..., 12, 0, -18]], dtype=object)

In [26]:
hash_table.shape

(2040722, 100, 32)

In [34]:
kmer_num = feature_matrix.shape[1]
hash_table=np.reshape(hash_table,(kmer_num,3200))

In [30]:
len(new_hash[1])

3200

In [18]:
def get_read_intervals(meta_df):
    read_intervals = {
        i: [GenomicInterval(strand, start, end)]
        for i, strand, start, end in zip(
            meta_df.index,
            meta_df["reference_strand"],
            meta_df["reference_start"],
            meta_df["reference_end"],
        )
    }
    return read_intervals

read_intervals = get_read_intervals(meta_df)

reference_graph = OverlapGraph.from_intervals(read_intervals)
nr_edges = set((node_1, node_2) for node_1, node_2, data in reference_graph.edges(data=True) if not data['redundant'])
connected_component_count = len(list(nx.connected_components(reference_graph)))
len(reference_graph.nodes), len(reference_graph.edges), len(nr_edges), connected_component_count

(12926, 324674, 25276, 2)

In [19]:
read_ids = np.array(list(read_features))

In [20]:
def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)
    binary_string = "{0:032b}".format(hash_value & 0xFFFFFFFF)  
    hash_array = np.array([int(x) for x in binary_string], dtype=np.int32) 
    hash_array = np.where(hash_array == 0, -1, 1)  
    return hash_array  

def mp_get_hashtable(  
    feature_matrix: list,   
    repeat: int,   
    seed: int,
    processes:int,) -> Mapping[int,list]:  
      
    rng = np.random.default_rng(seed)  
    hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeat, dtype=np.uint64)  
    kmer_num = feature_matrix.shape[1]
    hash_table = np.empty((kmer_num,repeat),dtype=object) 

    with sharedmem.MapReduce(np=processes) as pool:

        def work(i):
            seed = hash_seeds[i]
            result = np.empty(kmer_num, dtype=object) 
            for kmer_index in range(kmer_num):
                result[kmer_index]=_hash(kmer_index, seed=seed)
            return i,result

        def reduce(i, result):
            hash_table[:,i] = result            

        pool.map(work, range(repeat), reduce=reduce)
    return hash_table

def hamming_distance(x, y):  
    return np.count_nonzero(x != y)

## multi-process calculate simhash value
def mp_get_simhash(
    read_features: list,  
    feature_matrix: list,
    hash_table,
    tf:bool,
    idf:bool,
    processes:int,):
    
    if idf == True:
        nested_list = list(read_features.values())
        unrongh_nest = [list(set(sublist)) for sublist in nested_list]  
        merged_list = list(chain.from_iterable(unrongh_nest)) 
        count = Counter(merged_list)
        sorted_counts = dict(sorted(count.items(), key=lambda x: x[0]))  
        times = np.array(list(sorted_counts.values()))
        x = len(read_features)
        arr = np.full(feature_matrix.shape[1],x)  
        idf = np.log(arr/times)
        hash_table = hash_table*idf[:, np.newaxis]
    if tf == False:
        read_features = {k:list(set(v)) for k,v in read_features.items()} 

       
    reads_simhash_array = np.empty((len(read_features),hash_table.shape[1]*32),dtype=object)
    with sharedmem.MapReduce(np=processes) as pool:

        def work(i):
            read_kmer = list(read_features.values())[i]
            one_read_hash = np.sum(hash_table[read_kmer,:],axis=0)
            conc_hash = np.concatenate(one_read_hash)
            result = np.where(conc_hash > 0, 1, 0)
            return i,result

        def reduce(i, result):
            reads_simhash_array[i,:] = result            
        pool.map(work, range(len(read_features)), reduce=reduce)
    return reads_simhash_array

In [21]:
hash_table = mp_get_hashtable(feature_matrix,repeat =100, seed = 4829,processes=12)
print('hashtable established')

In [ ]:
mp_concat_simhash = mp_get_simhash(read_features,feature_matrix,hash_table,tf = None,idf = None,processes=8) 
nbrs = NearestNeighbors(n_neighbors=21, algorithm='auto', metric=hamming_distance)
nbrs.fit(mp_concat_simhash)  
indices = nbrs.kneighbors(mp_concat_simhash,return_distance=False)
nbr_indices = indices[:, 1:]

df_rows = []
for k in k_values:
    graph = OverlapGraph.from_neighbor_indices(
        neighbor_indices=nbr_indices,
        n_neighbors=k,
        read_ids=read_ids,
        require_mutual_neighbors=False,
    )
    graph_stats = get_overlap_statistics(query_graph=graph, reference_graph=reference_graph)
    stats = { "n_neighbors": k}
    stats = {"description":'SimHash_TF-IDF', "n_neighbors": k, "repeat_time": 100,
                **graph_stats}
    df_rows.append(stats)
df_tfidf = pd.DataFrame(df_rows)
df_tfidf.to_csv('/home/miaocj/docker_dir/kNN-overlap-finder/data/evaluation/human/HLA/ONT_R9/kmer_k16/SimHash_None_None_overlap_stat.tsv',sep='\t')

NameError: name 'mp_get_simhash' is not defined

In [16]:
df_tfidf

,description,n_neighbors,repeat_time,precision,nr_precision,recall,nr_recall,singleton_count,singleton_fraction,N50,N50_percentage
0,SimHash_TF-IDF,2,100,0.682640,0.206799,0.034578,0.173497,3033,0.326832,36,0.39%
1,SimHash_TF-IDF,3,100,0.668357,0.180317,0.049794,0.222505,2099,0.226185,138,1.49%
2,SimHash_TF-IDF,4,100,0.658011,0.163658,0.064472,0.265589,1576,0.169828,215,2.32%
3,SimHash_TF-IDF,5,100,0.649636,0.151263,0.078775,0.303799,1238,0.133405,218,2.35%
4,SimHash_TF-IDF,6,100,0.642609,0.140782,0.092881,0.337025,992,0.106897,231,2.49%
5,SimHash_TF-IDF,7,100,0.634307,0.131930,0.106191,0.365821,834,0.089871,234,2.52%
6,SimHash_TF-IDF,8,100,0.627851,0.125011,0.119391,0.393731,736,0.079310,275,2.96%
7,SimHash_TF-IDF,9,100,0.622290,0.118347,0.132434,0.417156,641,0.069073,301,3.24%
8,SimHash_TF-IDF,10,100,0.616903,0.112700,0.145260,0.439528,567,0.061099,304,3.28%
9,SimHash_TF-IDF,11,100,0.612117,0.108083,0.157788,0.461458,484,0.052155,305,3.29%


In [111]:
df_tfidf

,description,n_neighbors,repeat_time,precision,nr_precision,recall,nr_recall,singleton_count,singleton_fraction,N50,N50_percentage
0,SimHash_TF-IDF,2,10,0.682640,0.206799,0.034578,0.173497,3033,0.326832,36,0.39%
1,SimHash_TF-IDF,3,10,0.668357,0.180317,0.049794,0.222505,2099,0.226185,138,1.49%
2,SimHash_TF-IDF,4,10,0.658011,0.163658,0.064472,0.265589,1576,0.169828,215,2.32%
3,SimHash_TF-IDF,5,10,0.649636,0.151263,0.078775,0.303799,1238,0.133405,218,2.35%
4,SimHash_TF-IDF,6,10,0.642609,0.140782,0.092881,0.337025,992,0.106897,231,2.49%
5,SimHash_TF-IDF,7,10,0.634307,0.131930,0.106191,0.365821,834,0.089871,234,2.52%
6,SimHash_TF-IDF,8,10,0.627851,0.125011,0.119391,0.393731,736,0.079310,275,2.96%
7,SimHash_TF-IDF,9,10,0.622290,0.118347,0.132434,0.417156,641,0.069073,301,3.24%
8,SimHash_TF-IDF,10,10,0.616903,0.112700,0.145260,0.439528,567,0.061099,304,3.28%
9,SimHash_TF-IDF,11,10,0.612117,0.108083,0.157788,0.461458,484,0.052155,305,3.29%


In [ ]:
import numpy as np  
import collections  
from typing import Mapping  
from sklearn.neighbors import NearestNeighbors  
from scipy.spatial.distance import hamming  
import mmh3

for repeat in [100]:
    print(repeat)
    concat_simhash = _get_simhash(read_features,feature_matrix,repeat =repeat, seed = 4829,tf = True,idf = True)
    nbrs = NearestNeighbors(n_neighbors=21, algorithm='auto', metric=hamming_distance)
    nbrs.fit(concat_simhash)  
    indices = nbrs.kneighbors(concat_simhash,return_distance=False)
    nbr_indices = indices[:, 1:]

    for k in k_values:
        graph = OverlapGraph.from_neighbor_indices(
            neighbor_indices=nbr_indices,
            n_neighbors=k,
            read_ids=read_ids,
            require_mutual_neighbors=False,
        )
        graph_stats = get_overlap_statistics(query_graph=graph, reference_graph=reference_graph)
        stats = { "n_neighbors": k}
        stats = {"description":'SimHash', "n_neighbors": k, "repeat_time": repeat,
                    **graph_stats}
        df_rows.append(stats)
    df = pd.DataFrame(df_rows)

In [ ]:
repeat = 100

def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)
    binary_string = "{0:032b}".format(hash_value & 0xFFFFFFFF)  
    hash_array = np.array([int(x) for x in binary_string], dtype=np.int32) 
    hash_array = np.where(hash_array == 0, -1, 1)  
    return hash_array  

all_read_simhash = []
rng = np.random.default_rng(4829)  
hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeat, dtype=np.uint64)
kmer_num = feature_matrix.shape[1]
hash_table = np.empty((kmer_num,repeat),dtype=object)  
for flag,seed in enumerate(hash_seeds):
    print(flag)
    kmer_hash_indice = {} 
    for kmer_index in range(kmer_num):
        hash_table[kmer_index,flag]=_hash(kmer_index, seed=seed)
new_hash = hash_table*idf[:, np.newaxis]

for read_ind,read_kmer in read_features.items():
    one_read_hash = np.sum(new_hash[read_kmer,:],axis=0)
    conc_hash = np.concatenate(one_read_hash)
    simhash = np.where(conc_hash > 0, 1, 0)
    all_read_simhash.append(simhash)
reads_simhash_array = np.array(all_read_simhash)

In [ ]:
reads_simhash_array

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 0, 1, ..., 1, 1, 1],
       [0, 0, 0, ..., 1, 1, 1],
       ...,
       [1, 0, 0, ..., 1, 1, 1],
       [0, 1, 0, ..., 1, 1, 1],
       [1, 0, 1, ..., 1, 1, 1]])

In [ ]:
nbrs = NearestNeighbors(n_neighbors=21, algorithm='auto', metric=hamming_distance)
nbrs.fit(reads_simhash_array)  
indices = nbrs.kneighbors(reads_simhash_array,return_distance=False)
nbr_indices = indices[:, 1:]

read_ids = np.array(list(read_features))
k_values = np.arange(2, max_n_neighbors + 1)
df_rows = []

for k in k_values:
    graph = OverlapGraph.from_neighbor_indices(
        neighbor_indices=nbr_indices,
        n_neighbors=k,
        read_ids=read_ids,
        require_mutual_neighbors=False,
    )
    graph_stats = get_overlap_statistics(query_graph=graph, reference_graph=reference_graph)
    stats = { "n_neighbors": k}
    stats = {"description":'SimHash', "n_neighbors": k, "repeat_time": repeat,
                **graph_stats}
    df_rows.append(stats)
df = pd.DataFrame(df_rows)

In [ ]:
##带tf idf 初始版本
def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)
    binary_string = "{0:032b}".format(hash_value & 0xFFFFFFFF)  
    hash_array = np.array([int(x) for x in binary_string], dtype=np.int32) 
    hash_array = np.where(hash_array == 0, -1, 1)  
    return hash_array  

def _get_simhash(  
    read_features: list,  
    feature_matrix: list,   
    repeat: int,   
    seed: int,
    tf:bool,
    idf:bool,) -> Mapping[int,list]:  
      
    rng = np.random.default_rng(seed)  
    hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeat, dtype=np.uint64)  
  
    all_read_simhash = []
    kmer_num = feature_matrix.shape[1]
    hash_table = np.empty((kmer_num,repeat),dtype=object)  
    for flag,seed in enumerate(hash_seeds):
        for kmer_index in range(kmer_num):
            hash_table[kmer_index,flag]=_hash(kmer_index, seed=seed)

    if idf == True:
        nested_list = list(read_features.values())
        unrongh_nest = [list(set(sublist)) for sublist in nested_list]  
        merged_list = list(chain.from_iterable(unrongh_nest)) 
        count = Counter(merged_list)
        sorted_counts = dict(sorted(count.items(), key=lambda x: x[0]))  
        times = np.array(list(sorted_counts.values()))
        x = len(read_features)
        arr = np.full(feature_matrix.shape[1],x)  
        idf = np.log(arr/times)
        hash_table = hash_table*idf[:, np.newaxis]
    if tf == False:
        read_features = {k:set(v) for k,v in read_features}

    for read_kmer in read_features.values():
        one_read_hash = np.sum(hash_table[read_kmer,:],axis=0)
        conc_hash = np.concatenate(one_read_hash)
        simhash = np.where(conc_hash > 0, 1, 0)
        all_read_simhash.append(simhash)
    reads_simhash_array = np.array(all_read_simhash)
  
    return reads_simhash_array 

def hamming_distance(x, y):  
    return np.count_nonzero(x != y)

In [ ]:
new[3][1]

array([ 5.0724357,  5.0724357, -5.0724357, -5.0724357, -5.0724357,
       -5.0724357,  5.0724357, -5.0724357,  5.0724357,  5.0724357,
       -5.0724357,  5.0724357,  5.0724357,  5.0724357, -5.0724357,
       -5.0724357, -5.0724357, -5.0724357, -5.0724357,  5.0724357,
       -5.0724357,  5.0724357, -5.0724357, -5.0724357,  5.0724357,
        5.0724357, -5.0724357,  5.0724357,  5.0724357, -5.0724357,
       -5.0724357,  5.0724357])

In [ ]:
df

,description,n_neighbors,repeat_time,precision,nr_precision,recall,nr_recall,singleton_count,singleton_fraction,N50
0,SimHash,2,20,0.007934,0.000784,0.000784,0.000811,5104,0.999608,1
1,SimHash,3,20,0.008033,0.000784,0.001191,0.001216,5103,0.999412,1
2,SimHash,4,20,0.009650,0.000784,0.001908,0.001622,5102,0.999217,1
3,SimHash,5,20,0.010621,0.000784,0.002624,0.002027,5101,0.999021,1
4,SimHash,6,20,0.010583,0.000784,0.003138,0.002433,5100,0.998825,1
5,SimHash,7,20,0.010556,0.000784,0.003651,0.002838,5099,0.998629,1
6,SimHash,8,20,0.010512,0.000760,0.004154,0.003142,5098,0.998433,1
7,SimHash,9,20,0.010238,0.000762,0.004551,0.003548,5086,0.996083,1
8,SimHash,10,20,0.010234,0.000745,0.005055,0.003852,5085,0.995887,1
9,SimHash,11,20,0.009733,0.000749,0.005287,0.004257,5084,0.995691,1


In [ ]:
##32位，70个repeat
import numpy as np  
import collections  
from typing import Mapping  
from sklearn.neighbors import NearestNeighbors  
from scipy.spatial.distance import hamming  
import mmh3  

def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)  
    binary_string = "{0:032b}".format(hash_value & 0xFFFFFFFF)  
    hash_array = np.array([int(x) for x in binary_string], dtype=np.int32)  
    return hash_array  

def _get_simhash(  
    read_features: list,  
    feature_matrix: list,   
    repeat: int,   
    seed: int) -> Mapping[int,list]:  

    hash_seeds = np.array(range(repeat))
    kmer_num = feature_matrix.shape[1]
    kmer_index = np.array(range(kmer_num))

    table = []
    for seed in hash_seeds:
        prime1 = 2654435761  # A large prime number
        prime2 = 0x27d4eb2d  # Another large prime, often used in hashing
        hash_value = (kmer_index * prime1) ^ (seed * prime2)
        hash_t = hash_value % (2**32)
        binary_matrix = np.vectorize(np.binary_repr)(hash_t, width=32)
        one_repear_table = np.array([list(row) for row in binary_matrix.flatten()])
        table.append(one_repear_table.astype(int))
    hash_table = np.hstack(table)

    all_read_simhash = []
    for read_ind,read_kmer in read_features.items():
        one_read_hash = np.sum(hash_table[read_kmer,:],axis=0)
        simhash = np.where(conc_hash > 0, 1, 0)
        all_read_simhash.append(simhash)
    concat_simhash = np.array(all_read_simhash)
    return concat_simhash 

def hamming_distance(x, y):  
    return np.count_nonzero(x != y)

dfs = []
for repeat in range(20,110,10):
    print(repeat)
    concat_simhash = _get_simhash(read_features,feature_matrix,repeat =repeat, seed = 4829)
    nbrs = NearestNeighbors(n_neighbors=21, algorithm='auto', metric=hamming_distance)
    nbrs.fit(concat_simhash)  
    indices = nbrs.kneighbors(concat_simhash,return_distance=False)
    nbr_indices = indices[:, 1:]

    for k in k_values:
        graph = OverlapGraph.from_neighbor_indices(
            neighbor_indices=nbr_indices,
            n_neighbors=k,
            read_ids=read_ids,
            require_mutual_neighbors=False,
        )
        graphs[k] = graph

    df_rows = []
    for k in k_values:
        graph = graphs[k]
        graph_stats = get_overlap_statistics(query_graph=graph, reference_graph=reference_graph)
        stats = { "n_neighbors": k}
        stats = {"description":'SimHash', "n_neighbors": k, "repeat_time": repeat,
                    **graph_stats}
        df_rows.append(stats)
    df = pd.DataFrame(df_rows)
    dfs.append(df)
    
new = pd.concat(dfs)
new.to_csv('/home/miaocj/docker_dir/test_simhash.csv',sep='\t')

20
30
40
50
60
70
80
90
100


In [ ]:
new

,description,n_neighbors,repeat_time,precision,nr_precision,recall,nr_recall,singleton_count,singleton_fraction,N50
0,SimHash,2,20,0.007934,0.000784,0.000784,0.000811,5104,0.999608,1
1,SimHash,3,20,0.008033,0.000784,0.001191,0.001216,5103,0.999412,1
2,SimHash,4,20,0.009650,0.000784,0.001908,0.001622,5102,0.999217,1
3,SimHash,5,20,0.010621,0.000784,0.002624,0.002027,5101,0.999021,1
4,SimHash,6,20,0.010583,0.000784,0.003138,0.002433,5100,0.998825,1
...,...,...,...,...,...,...,...,...,...,...
14,SimHash,16,100,0.009649,0.000760,0.007621,0.006284,5075,0.993929,1
15,SimHash,17,100,0.009625,0.000762,0.008076,0.006690,5074,0.993733,1
16,SimHash,18,100,0.009571,0.000763,0.008502,0.007095,5073,0.993537,1
17,SimHash,19,100,0.009316,0.000764,0.008735,0.007501,5072,0.993341,1


In [ ]:
meta_df = pd.read_table(tsv_path).iloc[:MAX_SAMPLE_SIZE, :].reset_index()
read_indices = {read_name: read_id for read_id, read_name in meta_df['read_name'].items()}
feature_matrix = sp.sparse.load_npz(npz_path)[meta_df.index, :]

with gzip.open(json_path, "rt") as f:
    read_features = json.load(f)
    read_features = {i: read_features[i] for i in meta_df.index}

feature_weights = {i: 1 for i in range(feature_matrix.shape[1])}

In [ ]:
read_ids = np.array(list(read_features))
graphs = collections.defaultdict(dict)
k_values = np.arange(2, max_n_neighbors + 1)

for k in k_values:
    graph = OverlapGraph.from_neighbor_indices(
        neighbor_indices=nbr_indices,
        n_neighbors=k,
        read_ids=read_ids,
        require_mutual_neighbors=False,
    )
    graphs[k] = graph

In [ ]:
df_rows = []

for k in k_values:
    graph = graphs[k]
    graph_stats = get_overlap_statistics(query_graph=graph, reference_graph=reference_graph)
    stats = { "n_neighbors": k}
    stats = {"description":'SimHash', "n_neighbors": k, 
                 **graph_stats}
    df_rows.append(stats)
df = pd.DataFrame(df_rows)

In [ ]:
##70
df

,description,n_neighbors,precision,nr_precision,recall,nr_recall,singleton_count,singleton_fraction,N50
0,SimHash,2,0.691866,0.218421,0.056011,0.185080,1599,0.313161,27
1,SimHash,3,0.669721,0.195778,0.079271,0.242550,1080,0.211516,82
2,SimHash,4,0.651177,0.178502,0.100963,0.289682,787,0.154132,134
3,SimHash,5,0.634872,0.164610,0.121047,0.328502,624,0.122209,190
4,SimHash,6,0.620475,0.153457,0.140095,0.362660,487,0.095378,284
5,SimHash,7,0.605173,0.144046,0.157690,0.392864,407,0.079710,378
6,SimHash,8,0.591337,0.136604,0.174636,0.422258,343,0.067176,490
7,SimHash,9,0.578070,0.129028,0.190372,0.444760,296,0.057971,497
8,SimHash,10,0.565255,0.122479,0.205179,0.465335,259,0.050725,507
9,SimHash,11,0.552265,0.116772,0.218872,0.484391,225,0.044066,546


In [ ]:
##40
df

,description,n_neighbors,precision,nr_precision,recall,nr_recall,singleton_count,singleton_fraction,N50
0,SimHash,2,0.803557,0.288272,0.066944,0.251368,1041,0.203878,50
1,SimHash,3,0.790642,0.254084,0.096547,0.324752,587,0.114963,131
2,SimHash,4,0.778094,0.231235,0.124378,0.386884,364,0.071289,156
3,SimHash,5,0.767902,0.213350,0.151512,0.440604,234,0.045828,216
4,SimHash,6,0.757824,0.199063,0.177028,0.486722,147,0.028790,420
5,SimHash,7,0.748482,0.187767,0.201693,0.529597,109,0.021347,449
6,SimHash,8,0.739128,0.177327,0.225147,0.565376,76,0.014884,473
7,SimHash,9,0.730724,0.168622,0.247884,0.598723,56,0.010967,480
8,SimHash,10,0.721557,0.160861,0.269614,0.629130,45,0.008813,484
9,SimHash,11,0.713080,0.154311,0.290512,0.658017,31,0.006071,484


In [ ]:
##20
df

,description,n_neighbors,precision,nr_precision,recall,nr_recall,singleton_count,singleton_fraction,N50
0,SimHash,2,0.783025,0.274375,0.064862,0.237888,1133,0.221896,61
1,SimHash,3,0.769476,0.244558,0.093448,0.310866,662,0.129651,109
2,SimHash,4,0.754994,0.220402,0.120049,0.366815,416,0.081473,169
3,SimHash,5,0.745058,0.204536,0.146002,0.419522,271,0.053075,205
4,SimHash,6,0.732811,0.191511,0.170192,0.465538,192,0.037603,339
5,SimHash,7,0.722455,0.180849,0.193539,0.507095,142,0.027810,555
6,SimHash,8,0.711925,0.170326,0.215695,0.540138,125,0.024481,555
7,SimHash,9,0.702778,0.162199,0.237397,0.573485,93,0.018214,566
8,SimHash,10,0.694382,0.154631,0.258304,0.602068,75,0.014689,566
9,SimHash,11,0.685794,0.147694,0.278107,0.626900,54,0.010576,583


测试hash方法

In [ ]:
import hashlib

def simhash(features):
  
  # Generate a hash for each feature
  hashes = [hashlib.sha1(feature).hexdigest() for feature in features]
  
  # Combine the feature hashes to produce the final simhash
  concatenated_hash = ''.join(hashes)
  simhash = hashlib.sha1(concatenated_hash).hexdigest()
  
  return simhash

In [ ]:
x

In [ ]:
pip install xxhash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.3/194.3 kB 1.5 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import xxhash
x = np.random.rand(1024 * 1024 * 16)
h = xxhash.xxh64()
h.update(x); h.intdigest(); h.reset()

In [ ]:
x

array([0.29611066, 0.45210823, 0.21455805, ..., 0.22218214, 0.79713455,
       0.43898445])

In [ ]:
## 64位， 40个repeat
import numpy as np  
import collections  
from typing import Mapping  
from sklearn.neighbors import NearestNeighbors  
from scipy.spatial.distance import hamming  
import mmh3  

def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)  
    binary_string = "{0:064b}".format(hash_value & 0xFFFFFFFFFFFFFFFF)
    hash_array = np.array([int(x) for x in binary_string], dtype=np.int32)  
    return hash_array  

def _get_simhash(  
    read_features: list,  
    feature_matrix: list,   
    repeat: int,   
    seed: int) -> Mapping[int,list]:  
      
    rng = np.random.default_rng(seed)  
    hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeat, dtype=np.uint64)  
  
    repeat_all_read_hash = []  
    for s in hash_seeds:  
        print(s)
        kmer_hash_indice = {}  
        for kmer_index in range(feature_matrix.shape[1]):  
            kmer_index_str = str(kmer_index)  
            hash_list = _hash(kmer_index_str, seed=s)  
            kmer_hash_indice[kmer_index] = hash_list  
  
        all_read_simhash = []  
        for features in read_features.values():  
            feature_count = dict(collections.Counter(features))  
            one_read_hash = []  
            for indice, count in feature_count.items():  
                hash_list = kmer_hash_indice[indice]
                hash_list = np.where(hash_list == 0, -1, hash_list) 
                weighted_hash_list = hash_list * count  
                one_read_hash.append(weighted_hash_list)  
            one_read_hash_array = np.array(one_read_hash)  
            hash_sum = np.sum(one_read_hash_array, axis=0)   
            simhash_value = np.where(hash_sum > 0, 1, 0)  
            all_read_simhash.append(simhash_value)  
  
        repeat_all_read_hash.append(all_read_simhash)  
  
    concat_simhash = np.concatenate(repeat_all_read_hash, axis=1)  
  
    return concat_simhash 

concat_simhash = _get_simhash(read_features,feature_matrix,repeat = 40, seed = 4829)
def hamming_distance(x, y):  
    return np.count_nonzero(x != y)
nbrs = NearestNeighbors(n_neighbors=21, algorithm='auto', metric=hamming_distance)
nbrs.fit(concat_simhash)  
indices = nbrs.kneighbors(concat_simhash,return_distance=False)
nbr_indices = indices[:, 1:]


299897765
2378963497
44873935
2670060064
25935175
920060685
1586326711
1550303102
278283145
1103905279
2497529993
993549511
3538826777
827898949
1744767203
1454149367
347405931
293704892
739522939
2787976300
923371613
3028338653
1487664578
1269177166
1108403661
2333147459
3468047501
3767013626
3224645638
216444033
873431322
2428674822
205263707
3132866966
3313916789
964022396
3691436413
768347323
2334284253
3571135595


In [ ]:
rng = np.random.default_rng(seed)
beta = rng.uniform(0, 1, (feature_count, dimension_count))
x = rng.uniform(0, 1, (feature_count, dimension_count))
u1 = rng.uniform(0, 1, (feature_count, dimension_count))
u2 = rng.uniform(0, 1, (feature_count, dimension_count))

for j_sample in range(0, sample_count):
    feature_indices = sparse.find(data[:, j_sample] > 0)[0]
    gamma = -np.log(np.multiply(u1[feature_indices, :], u2[feature_indices, :]))
    t_matrix = np.floor(
        np.divide(
            matlib.repmat(
                np.log(data[feature_indices, j_sample].todense()),
                1,
                dimension_count,
            ),
            gamma,
        )
        + beta[feature_indices, :]
    )
    y_matrix = np.exp(np.multiply(gamma, t_matrix - beta[feature_indices, :]))
    a_matrix = np.divide(
        -np.log(x[feature_indices, :]),
        np.divide(y_matrix, u1[feature_indices, :]),
    )

In [ ]:
rng = np.random.default_rng(12)

In [ ]:
##建立hash矩阵
import numpy as np  
import collections  
from typing import Mapping  
from sklearn.neighbors import NearestNeighbors  
from scipy.spatial.distance import hamming  
import mmh3  

def _hash(kmer_index: int, seed: int) -> np.ndarray:  
    hash_value = mmh3.hash(str(kmer_index), seed=seed)  
    binary_string = "{0:032b}".format(hash_value & 0xFFFFFFFF)  
    hash_array = np.array([int(x) for x in binary_string], dtype=np.int32)  
    return hash_array  

def _get_simhash(  
    read_features: list,  
    feature_matrix: list,   
    repeat: int,   
    seed: int) -> Mapping[int,list]:  
      
    rng = np.random.default_rng(seed)  
    hash_seeds = rng.integers(low=0, high=2**32 - 1, size=repeat, dtype=np.uint64)  
  
    repeat_all_read_hash = []  
    hash_table = np.empty((repeat,feature_matrix.shape[1]))  
    for s in hash_seeds:  
        print(s)
        kmer_hash_indice = {}  
        for kmer_index in range(feature_matrix.shape[1]): 
            hash_table[s,kmer_index] = _hash(kmer_index, seed=s)
    for 
         
        all_read_simhash = []  
        for features in read_features.values():  
            feature_count = dict(collections.Counter(features))  
            one_read_hash = []  
            for indice, count in feature_count.items():  
                hash_list = kmer_hash_indice[indice]
                hash_list = np.where(hash_list == 0, -1, hash_list) 
                weighted_hash_list = hash_list * count  
                one_read_hash.append(weighted_hash_list)  
            one_read_hash_array = np.array(one_read_hash)  
            hash_sum = np.sum(one_read_hash_array, axis=0)   
            simhash_value = np.where(hash_sum > 0, 1, 0)  
            all_read_simhash.append(simhash_value)  
  
        repeat_all_read_hash.append(all_read_simhash)  
  
    concat_simhash = np.concatenate(repeat_all_read_hash, axis=1)  
  
    return concat_simhash 

concat_simhash = _get_simhash(read_features,feature_matrix,repeat = 70, seed = 4829)
def hamming_distance(x, y):  
    return np.count_nonzero(x != y)
nbrs = NearestNeighbors(n_neighbors=21, algorithm='auto', metric=hamming_distance)
nbrs.fit(concat_simhash)  
indices = nbrs.kneighbors(concat_simhash,return_distance=False)
nbr_indices = indices[:, 1:]

SyntaxError: invalid syntax (971857692.py, line 31)